In [ ]:
players = [Player(), Player()]

grids = [p.setup_ships() for p in players]
#grids = map(lambda p: p.setup_ships(), players)

In [196]:
class Player:
    """
    Contains three methods
        1. setup_ships: takes no parameters and returns a list of lists showing the start position
                        of showing the start positions of your ships
                        Block of row i and column j, ship label 1-5 & 0 means no ship

        2. take_turn:   returns
                        -- a list of (row, column) tuples describing your shots
                        -- eg: [(1, 1), (1, 3), (1, 5), (2, 1) (4, 5)]
                        -- Note: should be equal to the number of ships present (check function)
                        -- or a tuple that has a ship number as the first element and the direction
                        -- of its moving value
                        -- 0: up, 1: right, 2: down, 3: left  (dict)

                        TODO: Do not take more shots than the number of ships
                        TODO: Do not move the ships outside the grid or radioactive zone

        3. get_name:   returns a string that is the name of the Player

    """
    import random
    from collections import defaultdict

    def __init__(self):

        self.player_battleground = [[0 for i in range(10)] for j in range(10)]
        self.opponent_battleground = [[0 for i in range(10)] for j in range(10)]

        self.ships_dd = Player.defaultdict(int)
        self.status = []

        self.enemy_dict = {(i, j): 0 for i in range(10) for j in range(10)}

        # number of turns
        self.counter = 0

        # total number of hits
        self.hits = 0

    def setup_ships(self):
        '''

        :return: a 10x10 grid with the location of our ships
        '''
        grid = [[0 for i in range(10)] for j in range(10)]

        # ship numbers
        ship = [i for i in range(1, 6)]

        # randomly shuffle the ship list
        Player.random.shuffle(ship)

        # select direction
        # 1 - down, 2 - right
        direction = [1, 2]

        for ship_number in ship:
            control = True

            while control:
                # randomly select a point on the grid
                ship_start_position = self.random_position()

                # randomly select the direction to be placed
                orientation = Player.random.choice(direction)

                if self.vacant(ship_start_position, ship_number, grid, orientation):
                    for row in range(0, ship_number):
                        # print(f'{ship_start_position} ship : {ship_number}')
                        if orientation == 1:
                            grid[ship_start_position[0] + row][ship_start_position[1]] = ship_number
                        elif orientation == 2:
                            grid[ship_start_position[0]][ship_start_position[1] + row] = ship_number

                    control = False

        self.player_battleground = grid
        return grid

    def take_turn(self, history):
        '''

        :param history: records the game
                        list that contains one entry per turn
                        dict with 3 keys
                            - 'shots' : list of shots made in the turn
                            - 'hits' : an integer number of hits that your shots made
                            - 'incoming' : opponents list of shots, [] if moved

        :return: a list of tuple/s describing the shots or changed location
                depending on the strategy
        '''
        # last_shot = history[-1]['incoming']
        # TODO: use history to short circuit the take_turn function
        # TODO: consider if the opponent moves

        last_shot = []
        if self.counter > 0:
            last_shot = history[-1]['incoming']
        # last_shot = history[-1]['incoming'] if self.counter > 0 else []

        self.player_battleground = self.update_opponent_shots(self.player_battleground, last_shot)

        # counting the number of ships left
        self.ships_dd = Player.defaultdict(int)  # reset count every time
        # ships_dd = Player.defaultdict(int)

        # returns the dict with the number of ships and hits
        for row in self.player_battleground:
            for point in row:
                self.ships_dd[point] += point

        self.status = self.ship_status(self.ships_dd)

        # Count the number of ships remaining
        number_of_ships = len(self.status) if self.status else []

        player_shots = []
        if self.counter > 0:
            player_shots = history[-1]['shots']
        # player_shots = history[-1]['shots'] if self.counter > 0 else []
        if self.counter > 0:
            self.hits += history[-1]['hits']

        # updating the shots taken by the player
        # self.enemy_dict = self.update_enemy_dict(self.enemy_dict, player_shots)
        self.update_enemy_dict(player_shots)

        if self.counter > 0 and history[-1]['hits']:
            # If the Player hit in its last turn, update the minesweeper matrix
            # then choose to shoot at the highest probable locations
            turn_hits = history[-1]['hits']
            self.enemy_dict = self.minesweeper(self.enemy_dict, player_shots, turn_hits)
            shots_list = [number for number in self.shot_generator(self.enemy_dict, number_of_ships)]

        # elif self.counter > 0 or history[-1]['hits'] == 0:
        #     shots_list = [number for number in self.random_selection(self.enemy_dict, number_of_ships)]

        else:
            # In case of the first shot, hits = 0 and it will shoot randomly
            # However, if the Player has already had some hits, the player will
            # use the minesweeper matrx and try again randomly at the more probable locations
            shots_list = [number for number in self.random_selection(self.enemy_dict, self.hits, number_of_ships)]

        # update the number of turns
        self.counter += 1

        return shots_list

    def get_name(self):
        '''

        :return: string - name of the Player
        '''
        return "Syndicate_10"

    # function to select a point on the grid
    def random_position(self):
        return [Player.random.randint(0, 9), Player.random.randint(0, 9)]

    # function to check if the position is vacant
    def vacant(self, pos, ship, grid, orientation):
        # checking the direction to place the ship
        if orientation == 1:
            # checking if the ship will fit in the board
            if pos[0] + ship <= 10:
                for i in range(0, ship):
                    # checking if any other ship is already present
                    if grid[pos[0] + i][pos[1]] != 0:
                        return False
                return True
        elif orientation == 2:
            # checking if the ship will fit in the board
            if pos[1] + ship <= 10:
                for i in range(0, ship):
                    # checking if any other ship is already present
                    if grid[pos[0]][pos[1] + i] != 0:
                        return False
                return True
        return False

        # function to shoot randomly with a given number of ships

    def random_shots(self, num=5):
        for x in range(0, num):
            yield Player.random.randint(0, 9), Player.random.randint(0, 9)

    # function to update the player battleground with the incoming shots
    def update_opponent_shots(self, grid, incoming_shots):
        if incoming_shots:
            for x, y in incoming_shots:
                grid[x][y] = 7
        return grid

    # function that counts the number of ships and her ports hit
    def ship_status(self, ships_dd):
        status = []
        for ship in ships_dd.keys():
            # as long as the default dictionary has a key, the ship is breathing
            if ship in [1, 2, 3, 4, 5]:
                status.append((ship, ships_dd[ship] / (ship ** 2)))
        return status

    # function to update the opponent battleground with the shots taken
    def update_enemy_dict(self, player_shots):
        if player_shots:
            for i, j in player_shots:
                self.enemy_dict[(i, j)] = -1

    # minesweeper matrix update
    def minesweeper(self, enemy_dict, player_shots, turn_hits):

        # up, down, right, left, upright, downright, up-left, down-left
        # minesweeper = [(-1, 0), (1, 0), (0, 1), (0, -1), (-1, 1), (1, 1), (-1, -1), (1, -1)]
        # up, down, right, left
        minesweeper = [(-1, 0), (1, 0), (0, 1), (0, -1)]

        # updating the enemy dictionary
        # only if it has hit!
        for i, j in player_shots:
            # opponent_battleground[i][j] = -1
            for x, y in minesweeper:
                # checking if the next position is in the board
                if (x + i) >= 0 and (x + i) <= 9 and (y + j) >= 0 and (y + j) <= 9:
                    # checking if it has already not been hit
                    if enemy_dict[(i + x, j + y)] != -1:
                        enemy_dict[(i + x, j + y)] += turn_hits # TODO: check effectiveness

        return enemy_dict

    # shot generator based on the minesweeper
    def shot_generator(self, enemy_dict, num=5):
        sorted_shot_list = sorted([(i[1], i[0]) for i in enemy_dict.items()], reverse=True)
        for i in range(num):
            yield sorted_shot_list[i][1]

    # function that randomly selects positions to hit and removes them
    def random_selection(self, enemy_dict, hits, num=5):
        sorted_shot_list = sorted([(i[1], i[0]) for i in enemy_dict.items() if i[1] >= 0], reverse=True)

        # If the number of ships are greater than your shots space
        if num > len(sorted_shot_list):
            num = len(sorted_shot_list)

        # If there are existing hits, choose to select randomly from the higher weighted points
        if not hits:
            shot = Player.random.sample(sorted_shot_list, k=num)
        else:
            shot = Player.random.sample(sorted_shot_list[:7], k=num)

        shot_list = [i[1] for i in shot]

        return shot_list


In [188]:
class Player:
    """
    Contains three methods
        1. setup_ships: takes no parameters and returns a list of lists showing the start position
                        of showing the start positions of your ships
                        Block of row i and column j, ship label 1-5 & 0 means no ship

        2. take_turn:   returns
                        -- a list of (row, column) tuples describing your shots
                        -- eg: [(1, 1), (1, 3), (1, 5), (2, 1) (4, 5)]
                        -- Note: should be equal to the number of ships present (check function)
                        -- or a tuple that has a ship number as the first element and the direction
                        -- of its moving value
                        -- 0: up, 1: right, 2: down, 3: left  (dict)

        3. get_name:   returns a string that is the name of the Player

    """
    import random
    from collections import defaultdict

    def __init__(self):

        self.player_battleground = [[0 for i in range(10)] for j in range(10)]

        self.ships_dd = Player.defaultdict(int)
        self.status = []

        self.enemyport = [[0 for i in range(10)] for j in range(10)]

        # Set up the probability map.
        self.enemy_probmap = [[15 for i in range(10)] for j in range(10)]

        # TODO: Remove this
        self.potential_list = [(i, j) for i in range(10) for j in range(10)]

        # number of turns
        self.counter = 0

        self.positions = None  # store indexes for ships

    def setup_ships(self):
        '''

        :return: a 10x10 grid with the location of our ships
        '''
        grid = [[0 for i in range(10)] for j in range(10)]

        self.init_position()
        while self.is_good_position() == False:
            self.init_position()
            self.is_good_position()

        # update postion index to myport
        points = self.positions
        points_len = len(points)
        i = 0
        while i < points_len:
            for item in points[i]:
                row = item[0]
                col = item[1]
                grid[row][col] = i + 1
            i += 1

        self.player_battleground = grid
        return grid

    def take_turn(self, history):
        '''

        :param history: records the game
                        list that contains one entry per turn
                        dict with 3 keys
                            - 'shots' : list of shots made in the turn
                            - 'hits' : an integer number of hits that your shots made
                            - 'incoming' : opponents list of shots, [] if moved

        :return: a list of tuple/s describing the shots or changed location
                depending on the strategy
        '''
        # last_shot = history[-1]['incoming']
        # TODO: use history to short circuit the take_turn function

        last_shot = []
        if self.counter > 0:
            last_shot = history[-1]['incoming']

        # last_shot = history[-1]['incoming'] if self.counter > 0 else []

        self.player_battleground = self.update_opponent_shots(self.player_battleground, last_shot)

        # counting the number of ships left
        self.ships_dd = Player.defaultdict(int)  # reset count every time
        # ships_dd = Player.defaultdict(int)

        # returns the dict with the number of ships and hits
        for row in self.player_battleground:
            for point in row:
                self.ships_dd[point] += point

        self.status = self.ship_status(self.ships_dd)

        # Count the number of ships remaining
        number_of_ships = len(self.status)

        # Updating the opponents probability matrix
        self.update_enemy_probmap(history)

        # updating the enemy port
        previous_shots = []
        if self.counter > 0:
            previous_shots = history[-1]['shots']

        # previous_shots = history[-1]['shots'] if self.counter > 0 else []

        self.enemyport = self.update_enemy_map(self.enemyport, previous_shots)

        # Based on some logic here, choose to shoot
        # using probability

        shots_list = self.shot(number_of_ships)

        # update the number of turns
        self.counter += 1

        return shots_list

    def get_name(self):
        '''

        :return: string - name of the Player
        '''
        return "Syndicate_10"

    # function to shoot randomly with a given number of ships
    def random_shots(self, num=5):
        for x in range(0, num):
            yield Player.random.randint(0, 9), Player.random.randint(0, 9)

    # function to update the player battleground with the incoming shots
    def update_opponent_shots(self, grid, incoming_shots):
        if incoming_shots:
            for x, y in incoming_shots:
                grid[x][y] = 9
        return grid

    # function that counts the number of ships and her ports hit
    def ship_status(self, ships_dd):
        status = []
        for ship in ships_dd.keys():
            # as long as the default dictionary has a key, the ship is breathing
            if ship in [1, 2, 3, 4, 5]:
                status.append((ship, ships_dd[ship] / (ship ** 2)))
        return status

    def update_enemy_probmap(self, history):
        # reset enemy_probmap
        self.enemy_probmap = [[-1 for i in range(10)] for j in range(10)]

        total_hit = 0
        total_shot = 0
        for turn in history:
            # record the success rate of each shot in enemy_probmap
            if turn['shots']:
                prob = int(turn['hits'] / len(turn['shots']) * 100 + 0.5)
                for i in turn['shots']:
                    self.enemy_probmap[i[0]][i[1]] = prob
            total_hit += turn['hits']
            total_shot += len(turn['shots'])

        # update the probablity of spots that have not been shots.
        prob = int((15 - total_hit) / (100 - total_shot) * 100 + 0.5)
        for row in range(10):
            for column in range(10):
                if self.enemy_probmap[row][column] == -1:
                    self.enemy_probmap[row][column] = prob

    def update_enemy_map(self, enemyport, previous_shots):
        if previous_shots:
            for x, y in previous_shots:
                enemyport[x][y] = 9
        return enemyport

        # Function that generate the list of shots

    def shot(self, myship_number):
        """
        Store the "coordinates of spots that have not been shot"
        into the list called "targets" and shuffle them to add randomness
        """
        try:  # until the list index out of range error is not handled
            targets = []
            for row in range(10):
                for column in range(10):
                    if self.enemyport[row][column] == 0:
                        targets.append((row, column))
            Player.random.shuffle(targets)
            """
            Evaluate likelihood score for points in the "target" list
            In this code, likelihood score (LHS) defined as
            LHS = 2*P(point) + 0.9*P(one_neighbour) + 0.8*P(2setp_neighbours)
                where P = probability of presense of a shp
            Where neighbours are not available, P(point) replace P(neighbours)
            Note that LHS is not probability but probability utility.
            """
            scores = []
            for i in range(len(targets)):  # i represent target number
                # Calculate LHS of target point i
                score = 0
                for row_step in range(-2, 3):
                    #  Where neighbours are not available, P(point) replace P(neibours)
                    if targets[i][0] + row_step in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
                        prob = self.enemy_probmap[targets[i][0] + row_step][targets[i][1]]
                    else:
                        prob = self.enemy_probmap[targets[i][0]][targets[i][1]]
                    score += (1 - 0.1 * abs(row_step)) * prob

                for col_step in range(-2, 3):
                    #  Where neighbours are not available, P(point) replace P(neibours)
                    if targets[i][1] + col_step in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
                        prob = self.enemy_probmap[targets[i][0]][targets[i][1] + col_step]
                    else:
                        prob = self.enemy_probmap[targets[i][0]][targets[i][1]]
                    score += (1 - 0.1 * abs(col_step)) * prob
                # Store LHS and target point i and sort at the end
                scores.append((score, i))
            scores.sort(reverse=True)

            # Return the top LSH points as list of tuple
            # The number of returning points is the same as the number of our ship
            shot_list = []

            for j in range(myship_number):
                if j <= len(targets) - 1:
                    shot_row = targets[scores[j][1]][0]
                    shot_col = targets[scores[j][1]][1]
                else:
                    shot_row = targets[-1][0]
                    shot_col = targets[-1][1]
                shot_list.append((shot_row, shot_col))

        except Exception as e:
            shot_list = []
        return shot_list

    """ is good placing strategy if ships dont touch"""

    def is_good_position(self):
        points = self.positions
        signal = True
        points_len = len(points)
        for ship in points:
            position = points.index(ship)
            length = len(ship)
            # get two end points for the ship
            end = None
            if length == 1:
                end = ship
            else:
                end = [ship[0], ship[length - 1]]

            # check if end points touch any other ship points:
            i = position + 1
            while i < points_len:
                another_ship = points[i]
                for stuff in another_ship:
                    for item in end:
                        if (stuff[0] == item[0]) and (abs(stuff[1] - item[1])) == 2:
                            return False
                        elif (stuff[1] == item[1]) and (abs(stuff[0] - item[0])) == 2:
                            return False
                        else:
                            signal = True

                i += 1

        return signal

    def init_position(self):  # initiate potential positions to place ships
        """strategy is divide myport our 5*5 area so every area will contain at most 2 ships"""

        # the direction ships point : 0 - down; 1- right
        row_range = {1: (0, 5), 2: (0, 5), 3: (5, 10), 4: (5, 10)}  # store row index range for different areas
        col_range = {1: (0, 5), 2: (5, 10), 3: (0, 5), 4: (5, 10)}  # store col index range for different areas

        # always place 1 and 2 together so the distribution is even
        area_order = Player.random.sample(range(1, 5), 4)  # areas are labeled 1,2,3,4, this one generate an order

        # For ship 1 and 2 in area i
        area = area_order[0]
        row_edge = row_range[area]
        col_edge = col_range[area]
        row_1 = Player.random.sample(range(row_edge[0], row_edge[1]), 1)[0]
        col_1 = Player.random.sample(range(col_edge[0], col_edge[1]), 1)[0]
        points = [[(row_1, col_1)]]  # get the position for ship 1

        # get the position for ship 2
        while True:
            dirt = Player.random.randint(0, 1)
            if dirt == 0:  # down
                row_2 = Player.random.sample(range(row_edge[0], row_edge[1] - 1), 1)[0]
                col_2 = Player.random.sample(range(col_edge[0], col_edge[1]), 1)[0]
                points_2 = [(row_2, col_2), (row_2 + 1, col_2)]
                if points[0][0] not in points_2:
                    points.append(points_2)

                    break

            else:  # right
                row_2 = Player.random.sample(range(row_edge[0], row_edge[1]), 1)[0]
                col_2 = Player.random.sample(range(col_edge[0], col_edge[1] - 1), 1)[0]
                points_2 = [(row_2, col_2), (row_2, col_2 + 1)]
                if points[0][0] not in points_2:
                    points.append(points_2)

                    break

        # For ship 3,4,5 in different areas:
        i = 3
        while i <= 5:
            area = area_order[i - 2]
            row_edge = row_range[area]
            col_edge = col_range[area]

            dirt = Player.random.randint(0, 1)
            if dirt == 0:  # down
                row_i = Player.random.sample(range(row_edge[0], row_edge[1] - i + 1), 1)[0]
                col_i = Player.random.sample(range(col_edge[0], col_edge[1]), 1)[0]
                points_i = [(row_i, col_i)]
                j = 1
                while j < i:
                    points_i.append((row_i + j, col_i))
                    j += 1

            else:  # right
                row_i = Player.random.sample(range(row_edge[0], row_edge[1]), 1)[0]
                col_i = Player.random.sample(range(col_edge[0], col_edge[1] - i + 1), 1)[0]
                points_i = [(row_i, col_i)]
                j = 1
                while j < i:
                    points_i.append((row_i, col_i + j))
                    j += 1
            points.append(points_i)
            i += 1

        self.positions = points
        return None


In [22]:
g.grids

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 4, 4, 4, 4, 0, 0, 0],
  [0, 0, 0, 5, 0, 0, 0, 0, 2, 2],
  [0, 0, 0, 5, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 5, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 5, 0, 0, 0, 0, 3, 0],
  [0, 1, 0, 5, 0, 0, 0, 0, 3, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 4, 4, 4, 4, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 5, 5, 5, 5, 5, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 2, 0, 0],
  [3, 3, 3, 0, 0, 0, 0, 2, 0, 0]]]

In [197]:
class Game():
    
    import random
    from collections import defaultdict

    def __init__(self):
        
        self.histories = [[], []]
        self.grids = [player0.setup_ships(), player1.setup_ships()]
        

    def run_game(self):
        
        histories = [[], []]

        counter = 0
        while counter < 20 and (counter == 0 or (len(player0.status) > 0 and len(player0.status) > 0)) :
            # Player 0 takes a turn
            turn_0 = player0.take_turn(histories[0])
            
            # Player 1 takes a turn
            turn_1 = player1.take_turn(histories[1])
            
            history_0, history_1 = update_history(histories[0], histories[1], self.grids[0], self.grids[1], turn_0, turn_1)
            if counter == 0:
                histories[0] = history_0
                histories[1] = history_1
            else:
                histories[0].extend(history_0)
                histories[1].extend(history_1)
                
            for i in range(10):
                print(player0.player_battleground[i])
            print('--------------------------')
            print(player0.status)
            print('--------------------------')
            
            print(sorted([(i[1], i[0]) for i in player1.enemy_dict.items()], reverse=True)[:10])
            print('--------------------------')
            print('--------------------------')
            
            counter += 1
            
        if len(player0.status) > 0:
            return 1
        if len(player1.status) > 1:
            return 2
            

In [23]:
player0.take_turn([])

[(7, 0), (6, 2), (9, 6), (5, 7), (5, 6)]

In [32]:
def update_history(history_0, history_1, grid_0, grid_1, turn_0, turn_1):
    
    hits_0 = count_hits(grid_1, turn_0)
    hits_1 = count_hits(grid_0, turn_1)
    
    history_0.append({'shots':turn_0, 'hits':hits_0, 'incoming':turn_1})
    history_1.append({'shots':turn_1, 'hits':hits_1, 'incoming':turn_0})
    
    return history_0, history_1

In [33]:
def count_hits(grid, incoming_shots):
    hits = 0
    for x, y in incoming_shots:
        if grid[x][y] in [1,2,3,4,5]:
            hits += 1
    return hits

In [37]:
update_history([], [], player0.setup_ships(), player1.setup_ships(), player0.take_turn([]), player1.take_turn([]))

([{'shots': [(8, 8), (5, 2), (8, 6), (7, 8), (6, 4)],
   'hits': 3,
   'incoming': [(3, 3), (5, 2), (0, 9), (5, 4), (0, 8)]}],
 [{'shots': [(3, 3), (5, 2), (0, 9), (5, 4), (0, 8)],
   'hits': 0,
   'incoming': [(8, 8), (5, 2), (8, 6), (7, 8), (6, 4)]}])

In [41]:
len(player0.status)

5

In [198]:
player0 = Player()

In [199]:
player1 = Player()

In [200]:
g = Game()

In [201]:
g.run_game()

[0, 0, 3, 3, 3, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 5, 5, 5, 5, 5, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 4, 4, 4, 4, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 2, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------
[(3, 1.0), (1, 1.0), (5, 1.0), (4, 1.0), (2, 1.0)]
--------------------------
[(0, (9, 9)), (0, (9, 8)), (0, (9, 7)), (0, (9, 6)), (0, (9, 5)), (0, (9, 4)), (0, (9, 3)), (0, (9, 2)), (0, (9, 1)), (0, (9, 0))]
--------------------------
--------------------------
[0, 0, 3, 3, 3, 0, 7, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 5, 5, 5, 5, 5, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 7, 0, 0, 4, 4, 4, 4, 0]
[0, 0, 0, 0, 0, 0, 7, 0, 0, 0]
[0, 0, 0, 0, 7, 0, 0, 0, 0, 0]
[2, 2, 7, 0, 0, 0, 0, 0, 0, 0]
--------------------------
[(3, 1.0), (1, 1.0), (5, 1.0), (4, 1.0), (2, 1.0)]
------------

TypeError: '>' not supported between instances of 'list' and 'int'

In [125]:
player0.status

[(1, 1.0), (5, 0.4), (4, 1.0)]

In [126]:
player1.status

[(4, 1.0), (3, 1.0)]

In [127]:
player0.player_battleground

[[0, 0, 0, 9, 9, 9, 9, 9, 9, 0],
 [0, 0, 0, 9, 9, 9, 9, 9, 9, 0],
 [0, 0, 0, 0, 9, 0, 9, 9, 9, 0],
 [0, 0, 0, 0, 0, 0, 0, 9, 9, 0],
 [9, 9, 9, 0, 0, 0, 0, 0, 0, 0],
 [9, 9, 9, 9, 0, 0, 0, 0, 0, 0],
 [9, 9, 9, 0, 0, 1, 0, 0, 0, 0],
 [9, 9, 9, 5, 5, 0, 0, 0, 0, 0],
 [9, 9, 9, 0, 0, 0, 4, 4, 4, 4],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0]]